# TenSEAL Introduction

[TenSEAL](https://github.com/OpenMined/TenSEAL) is a library for doing homomorphic encryption operations on tensors, built on top of Microsoft SEAL. It provides ease of use through a Python API, while preserving efficiency by implementing most of its operations using C++.

Homomorphic Encryption  is an encryption technique that allows computations to be made on ciphertexts and generates results that when decrypted, correspond to the results of the same computations made on plaintexts. 

**TenSEAL** supports *addition, substraction and multiplication* of encrypted vectors of either 
1. **Integers** - using **[BFV - Brakerski-Fan-Vercauteren](https://www.inferati.com/blog/fhe-schemes-bfv)** which is the secong generation of FHE and is based on the RLWE - Ring-Learning with Errors. BFV is instantiated over two rings: 1) the plaintext ring which includes encodings of unencrypted or intelligible messages and 2) the ciphertext ring which includes encrypted messages. Similar to any other FHE scheme, BFV allows an untrusted party to induce meaningful computation over encrypted data without access to the decryption key. This is possible due to the homomorphism property which offers a map (or function) between the plaintext and ciphertext spaces that preserves the operations in these two spaces.
2. **Real numbers** - using **[CKKS - Cheon-Kim-Kim-Song](https://www.inferati.com/blog/fhe-schemes-ckks)** which is Homomorphic Encryption for Arithmetic of Approximate Numbers (HEAAN), was proposed to offer homomorphic computation on real numbers. The main idea is to consider the noise, a.k.a. error $ e$ , which is introduced in Ring-Learning with Errors (Ring-LWE) based FHE schemes for security purposes, as part of the message $\mu $ (which we call here payload) we want to encrypt. The payload and the noise are combined to generate the plaintext ($\mu + e$) that we encrypt.

Next we will look at the most important object of the library, the TenSEALContext.

## BFV(Brakerski-Fan-Vercauteren) Scheme

#### TenSEAL Context

The *TenSEALContext* is a special object that holds different encryption keys and parameters , so that we only need to use a single object to make your encrypted computation instead of managing all the keys and the HE details. Basically, start by creating a single **TenSEALContext** before getting into encrypted computation.


In [1]:
import tenseal as ts

We need to specify the Homomorphic Encryption scheme- BFV that we want to apply here and its parameters.
1. `poly_modulus_degree` can be any power of 2 and is defined as the number
of slots per plain text of elements to be encoded in single ciphertext in a
2 by n/2 rectangular matrix.
2. `plain_modulus` that must be prime such that t-1 be divisible by 2^N.

In [2]:
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
context

**TenSEALContext** is now holding the secret key and we can decrypt without the need to provide it, however, we can choose to manage it as a separate object and will need to pass it to functions that require the secret key.

In [3]:
public_context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))

sk = public_context.secret_key()

# the context will drop the secret-key at this point
public_context.make_context_public()
print("Secret-key dropped")
print("Is the context private?", ("Yes" if public_context.is_private() else "No"))
print("Is the context public?", ("Yes" if public_context.is_public() else "No"))

Is the context private? Yes
Is the context public? No
Secret-key dropped
Is the context private? No
Is the context public? Yes


Let's fetch the secret key from the `public_context` and see if that raises an error.

## Encryption

In [4]:
# Let's create an encrypted vector of integers
plain_vector = [16, 25, 34, 43, 52, 70]
encrypted_vector = ts.bfv_vector(context, plain_vector)
print("The size of the encryted vector is", encrypted_vector.size())
encrypted_vector

The size of the encryted vector is 6


### ciphertext to plaintext (c2p) 

*Ciphertext*, or encrypted text, is a series of randomized letters and numbers which humans cannot make any sense of. An encryption algorithm takes in a plaintext message, runs the algorithm on the plaintext, and produces a ciphertext. The ciphertext can be reversed through the process of decryption, to produce the original plaintext.
And *Plaintext* can refer to anything which humans can understand and/or relate to. This may be as simple as English sentences, a script, or Java code. If you can make sense of what is written, then it is in plaintext.


#### Addition

In [5]:
add_result = encrypted_vector + [1, 2, 3, 4, 5, 6]
print(add_result.decrypt())

[17, 27, 37, 47, 57, 76]


#### Subtraction

In [6]:
sub_result = encrypted_vector - [1, 2, 3, 4, 5, 6]
print(sub_result.decrypt())

[15, 23, 31, 39, 47, 64]


#### Multiplication

In [7]:
mul_result = encrypted_vector * [1, 2, 3, 4, 5, 6]
print(mul_result.decrypt())

[16, 50, 102, 172, 260, 420]


## Evaluation

### ciphertext to ciphertext (c2c) evaluations

#### Addition

In [8]:
encrypted_add = add_result + sub_result
print(encrypted_add.decrypt())

[32, 50, 68, 86, 104, 140]


#### Subtraction

In [9]:
encrypted_sub = encrypted_add - encrypted_vector
print(encrypted_sub.decrypt())

[16, 25, 34, 43, 52, 70]


#### Multiplication

In [10]:
encrypted_mul = encrypted_add * encrypted_sub
print(encrypted_mul.decrypt())

[512, 1250, 2312, 3698, 5408, 9800]


We just made both ciphertext to plaintext (c2p) and ciphertext to ciphertext (c2c) evaluations (add, sub and mul). 

NOTE : An important thing to note is that you should never encrypt your plaintext values to evaluate them with ciphertexts if they don't need to be kept private. That's because c2p evaluations are more efficient than c2c.

## Conclusion and Future Work

This TenSEAL introduction notebook helps us understand basic functionalities like addition, subtraction and multiplication in ciphertext2plaintext and ciphertext2ciphertext. This tool is easy to understand and use as compared to other python libraries in the opensource world. We will further see how we can try out benchmarking and see how long it takes for various operations to perform and also see how Machine Learning can be applied to the encrypted data. 